<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-10 11:36:47
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.41 C
-------------------
Today PnL: 66.21 K (0.47%)
Current PnL: -28.17 L (-17.97%)
CY Booked + Current PnL: -13.43 L (-8.56%)
-------------------
Total profit:  1.59 L
Total loss:  -29.76 L
-------------------
Total Booked + Current PnL: 13.55 L (10.44%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.43%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.56 L (66.2%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.41%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.92,2.59,18.50,21.57,18595.0,2535.0,100515.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.68,XR,ATH,FINANCE
18,COALINDIA,-0.08,6.07,12.40,19.23,19790.0,9140.0,159600.0,484.83,24.27,54.0,L-LC,12.33,182.0,0.46,1.13,28.64,XY25,ATH,MINING
77,TTKPRESTIG,0.30,-24.64,32.83,0.09,24931.0,-24836.0,75941.0,770.00,76.31,47.0,M-SC,8.43,253.0,-1.00,0.54,4.52,OX40N,NTT,DURABLES
5,ANGELONE,2.59,14.07,8.88,24.20,30921.0,42962.0,348212.0,3033.00,62.46,63.0,X-SC,2.41,99.0,1.39,2.46,45.84,X40N,NTT,FINANCE
51,MEDANTA,-1.32,-4.34,29.71,24.08,36078.0,-5516.0,121434.0,1486.00,-12.07,53.0,X-SC,11.46,91.0,-0.15,0.86,11.22,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,JCHAC,7.65,-37.76,60.65,-0.01,48246.0,-48257.0,79548.0,2282.00,16182.61,54.0,M-SC,17.08,235.0,-1.00,0.56,10.29,OX40N,BTT,AC
7,ATULAUTO,6.55,-10.87,63.31,45.57,111243.0,-21420.0,175712.0,844.00,3813.51,77.0,M-SC,7.88,245.0,-0.19,1.24,32.97,XY24,NTT,AUTO
61,ROUTE,5.24,-53.24,261.50,69.05,162721.0,-70840.0,62226.0,2227.21,-62.09,49.0,H-SC,25.89,140.0,-0.44,0.44,7.25,SR,ATH,IT
14,BSOFT,4.62,-15.56,78.57,50.79,93314.0,-21887.0,118766.0,831.70,8.23,62.0,H-SC,7.30,171.0,-0.23,0.84,38.28,XR,ATH,IT
12,BATAINDIA,3.92,-40.73,128.10,35.20,97701.0,-52401.0,76269.0,2096.00,-0.07,60.0,X-SC,21.05,93.0,-0.54,0.54,8.39,X40,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,-3.89,-6.41,44.92,35.63,88105.0,-13436.0,196138.0,1286.17,-19.40,46.0,H-MC,6.16,119.0,-0.15,1.39,9.19,AR,ATH,PHARMA
76,TRIDENT,-2.93,-26.37,74.55,28.51,50638.0,-24330.0,67925.0,48.00,-12.81,54.0,M-SC,9.13,226.0,-0.48,0.48,16.23,XR,NTT,TEXTILES
79,VAIBHAVGBL,-1.81,-21.53,104.88,60.77,150424.0,-39350.0,143425.0,521.00,65.29,62.0,H-SC,3.01,158.0,-0.26,1.02,30.03,XR,NTT,JEWELLERY
51,MEDANTA,-1.32,-4.34,29.71,24.08,36078.0,-5516.0,121434.0,1486.00,-12.07,53.0,X-SC,11.46,91.0,-0.15,0.86,11.22,XY24,NTT,HEALTHCARE
13,BERGEPAINT,-1.19,-17.77,47.31,21.14,88445.0,-40391.0,186948.0,680.00,-23.58,27.0,X-MC,9.44,75.0,-0.46,1.32,0.00,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,3.07,3.48,100.92,107.91,154185.0,5135.0,152779.0,1641.55,-15.69,69.0,M-SC,10.94,220.0,0.03,1.08,14.94,OX40N,ATH,CABLES
66,SIS,0.06,-20.58,52.69,21.26,46462.0,-22853.0,88179.0,528.00,2084.97,66.0,H-SC,4.39,163.0,-0.49,0.62,21.56,OX40N,NTT,MISC
72,TATAELXSI,2.20,-22.92,71.30,32.03,72449.0,-30223.0,101612.0,9161.00,-16.54,48.0,H-SC,6.21,157.0,-0.42,0.72,13.81,OX40N,NTT,IT
77,TTKPRESTIG,0.30,-24.64,32.83,0.09,24931.0,-24836.0,75941.0,770.00,76.31,47.0,M-SC,8.43,253.0,-1.00,0.54,4.52,OX40N,NTT,DURABLES
40,INDIGOPNTS,0.16,-26.02,35.19,0.02,45446.0,-45413.0,129146.0,1408.00,96.99,38.0,M-SC,6.10,240.0,-1.00,0.91,12.33,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.08,6.07,12.40,19.23,19790.0,9140.0,159600.0,484.83,24.27,54.0,L-LC,12.33,182.0,0.46,1.13,28.64,XY25,ATH,MINING
50,MASFIN,0.92,2.59,18.50,21.57,18595.0,2535.0,100515.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.68,XR,ATH,FINANCE
26,FINCABLES,3.07,3.48,100.92,107.91,154185.0,5135.0,152779.0,1641.55,-15.69,69.0,M-SC,10.94,220.0,0.03,1.08,14.94,OX40N,ATH,CABLES
1,ABB,0.12,6.83,36.26,45.57,101339.0,17861.0,279480.0,7934.00,-31.40,73.0,H-MC,3.61,121.0,0.18,1.98,24.11,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,0.12,6.83,36.26,45.57,101339.0,17861.0,279480.0,7934.00,-31.40,73.0,H-MC,3.61,121.0,0.18,1.98,24.11,AR,NTT,ELECTRICAL
50,MASFIN,0.92,2.59,18.50,21.57,18595.0,2535.0,100515.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.68,XR,ATH,FINANCE
26,FINCABLES,3.07,3.48,100.92,107.91,154185.0,5135.0,152779.0,1641.55,-15.69,69.0,M-SC,10.94,220.0,0.03,1.08,14.94,OX40N,ATH,CABLES
39,INDIAMART,0.98,-2.44,111.88,106.72,134626.0,-3005.0,120331.0,4810.62,-51.77,56.0,H-SC,7.87,138.0,-0.02,0.85,20.37,AR,ATH,MISC
84,WIPRO,0.83,-4.84,81.03,72.27,130653.0,-8208.0,161240.0,420.00,-18.33,38.0,M-LC,3.97,101.0,-0.06,1.14,3.64,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.19,-17.77,47.31,21.14,88445.0,-40391.0,186948.0,680.00,-23.58,27.0,X-MC,9.44,75.0,-0.46,1.32,0.00,XY24,NTT,PAINTS
6,ASIANPAINT,-0.91,-16.27,44.45,20.95,93702.0,-40964.0,210804.0,3460.25,-17.08,28.0,X-LC,8.45,36.0,-0.44,1.49,13.57,X40,ATH,PAINTS
42,INFY,0.61,2.54,30.92,34.24,99204.0,7954.0,320842.0,1972.00,-20.32,32.0,X-LC,5.35,6.0,0.08,2.27,11.03,X40,NTT,IT
8,AWL,-0.59,-31.94,130.96,57.19,290678.0,-104168.0,221959.0,485.00,-67.84,34.0,X-MC,16.99,58.0,-0.36,1.57,1.16,XY24,NTT,FMCG
33,HCLTECH,-0.71,3.23,19.96,23.83,49848.0,7816.0,249740.0,1908.19,7.67,36.0,X-LC,6.90,13.0,0.16,1.77,19.82,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,1.29,-17.89,44.38,18.55,137829.0,-67672.0,310565.0,4311.59,-29.13,39.0,X-LC,1.19,3.0,-0.49,2.20,5.66,X40,ATH,IT
81,VBL,0.38,-7.26,46.29,35.67,135586.0,-22936.0,292906.0,671.64,-18.27,44.0,X-LC,1.25,5.0,-0.17,2.07,5.59,X40N,ATH,FMCG
21,DABUR,-0.63,-0.06,43.79,43.70,108559.0,-160.0,247908.0,735.00,-8.33,51.0,X-MC,1.87,73.0,-0.00,1.75,14.98,XY24,BTT,FMCG
5,ANGELONE,2.59,14.07,8.88,24.20,30921.0,42962.0,348212.0,3033.00,62.46,63.0,X-SC,2.41,99.0,1.39,2.46,45.84,X40N,NTT,FINANCE
34,HINDUNILVR,0.04,-5.24,19.96,13.67,48621.0,-13473.0,243590.0,2922.00,-12.27,61.0,X-LC,2.48,9.0,-0.28,1.72,15.00,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BERGEPAINT,-1.19,-17.77,47.31,21.14,88445.0,-40391.0,186948.0,680.00,-23.58,27.0,X-MC,9.44,75.0,-0.46,1.32,0.00,XY24,NTT,PAINTS
8,AWL,-0.59,-31.94,130.96,57.19,290678.0,-104168.0,221959.0,485.00,-67.84,34.0,X-MC,16.99,58.0,-0.36,1.57,1.16,XY24,NTT,FMCG
2,ABBOTINDIA,-0.33,-8.12,28.50,18.07,46837.0,-14515.0,164340.0,35195.00,-21.26,46.0,X-MC,9.61,64.0,-0.31,1.16,2.83,X40,ATH,PHARMA
53,PGHH,0.71,-10.11,46.86,32.01,89941.0,-21584.0,191936.0,17616.87,-36.80,48.0,X-MC,8.90,60.0,-0.24,1.36,3.74,X40,ATH,FMCG
3,ACC,-0.24,-28.47,129.49,64.15,220392.0,-67751.0,170200.0,3906.00,-58.06,49.0,X-SC,7.63,84.0,-0.31,1.20,4.03,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.83,-28.30,99.21,42.84,46244.0,-18394.0,46612.0,424.00,-53.17,59.0,X-SC,20.42,83.0,-0.40,0.33,12.12,XY24,NTT,MISC
58,RELAXO,0.46,-51.72,215.49,52.31,151009.0,-75083.0,70077.0,1176.00,-48.60,43.0,X-SC,13.07,92.0,-0.50,0.50,4.21,X40N,NTT,FOOTWEAR
12,BATAINDIA,3.92,-40.73,128.10,35.20,97701.0,-52401.0,76269.0,2096.00,-0.07,60.0,X-SC,21.05,93.0,-0.54,0.54,8.39,X40,NTT,FOOTWEAR
51,MEDANTA,-1.32,-4.34,29.71,24.08,36078.0,-5516.0,121434.0,1486.00,-12.07,53.0,X-SC,11.46,91.0,-0.15,0.86,11.22,XY24,NTT,HEALTHCARE
35,HONAUT,-1.05,-21.04,81.52,43.32,104835.0,-34272.0,128600.0,58357.33,-32.13,43.0,X-SC,9.76,90.0,-0.33,0.91,4.42,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.92,-20.88,40.02,10.78,76090.0,-50189.0,190129.0,452.00,-53.00,44.0,X-LC,18.81,1.0,-0.66,1.35,4.72,X40,NTT,FMCG
75,TMPV,0.66,-22.14,57.94,22.98,101913.0,-50004.0,175894.0,600.00,-19.41,66.0,X-LC,3.91,2.0,-0.49,1.24,12.45,XY24,NTT,AUTO
73,TCS,1.29,-17.89,44.38,18.55,137829.0,-67672.0,310565.0,4311.59,-29.13,39.0,X-LC,1.19,3.0,-0.49,2.20,5.66,X40,ATH,IT
81,VBL,0.38,-7.26,46.29,35.67,135586.0,-22936.0,292906.0,671.64,-18.27,44.0,X-LC,1.25,5.0,-0.17,2.07,5.59,X40N,ATH,FMCG
42,INFY,0.61,2.54,30.92,34.24,99204.0,7954.0,320842.0,1972.00,-20.32,32.0,X-LC,5.35,6.0,0.08,2.27,11.03,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.25,-31.67,94.48,32.89,49844.0,-24450.0,52756.0,1800.00,-751.34,58.0,L-MC,11.48,259.0,-0.49,0.37,45.29,XR,NTT,BANKS
67,SONACOMS,0.05,-8.17,51.46,39.09,47806.0,-8260.0,92899.0,804.02,-28.23,67.0,M-SC,2.31,227.0,-0.17,0.66,31.42,AR,ATH,AUTO
50,MASFIN,0.92,2.59,18.50,21.57,18595.0,2535.0,100515.0,397.04,-11.47,62.0,H-SC,3.64,168.0,0.14,0.71,46.68,XR,ATH,FINANCE
14,BSOFT,4.62,-15.56,78.57,50.79,93314.0,-21887.0,118766.0,831.70,8.23,62.0,H-SC,7.30,171.0,-0.23,0.84,38.28,XR,ATH,IT
79,VAIBHAVGBL,-1.81,-21.53,104.88,60.77,150424.0,-39350.0,143425.0,521.00,65.29,62.0,H-SC,3.01,158.0,-0.26,1.02,30.03,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ANGELONE,2.59,14.07,8.88,24.20,30921.0,42962.0,348212.0,3033.00,62.46,63.0,X-SC,2.41,99.0,1.39,2.46,45.84,X40N,NTT,FINANCE
7,ATULAUTO,6.55,-10.87,63.31,45.57,111243.0,-21420.0,175712.0,844.00,3813.51,77.0,M-SC,7.88,245.0,-0.19,1.24,32.97,XY24,NTT,AUTO
67,SONACOMS,0.05,-8.17,51.46,39.09,47806.0,-8260.0,92899.0,804.02,-28.23,67.0,M-SC,2.31,227.0,-0.17,0.66,31.42,AR,ATH,AUTO
11,BANDHANBNK,2.45,-20.13,140.91,92.42,313516.0,-56065.0,222494.0,400.00,56.61,71.0,H-SC,8.81,169.0,-0.18,1.57,28.58,XY24,NTT,BANKS
1,ABB,0.12,6.83,36.26,45.57,101339.0,17861.0,279480.0,7934.00,-31.40,73.0,H-MC,3.61,121.0,0.18,1.98,24.11,AR,NTT,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.30
1,25,44.41
2,50,76.28


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.93
MC    30.58
LC    23.49
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.58
X40      23.20
X40N     15.22
XR        8.96
AR        8.91
XY25      8.67
OX40N     7.54
SR        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.98
H-SC    23.04
X-LC    20.12
M-SC    11.70
X-SC    10.39
H-MC     4.84
M-MC     1.39
M-LC     1.14
L-LC     1.13
H-LC     1.10
L-SC     0.80
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.87,-12.66,50.52
FINANCE,13.46,-9.82,53.04
IT,12.26,-23.49,85.18
MISC,7.13,-28.78,81.46
ELECTRICAL,6.04,-9.95,50.07
PAINTS,5.07,-28.63,47.74
INSURANCE,4.71,-1.92,36.80
PHARMA,4.00,-3.89,36.59
AUTO,3.14,-17.93,58.71


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3211961.0,21
AR,1312854.0,10
XR,1225773.0,12
X40,1219766.0,15
X40N,972864.0,10
OX40N,726881.0,10
XY25,395071.0,6
SR,290532.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3537959.0,24
X-MC,1639533.0,16
M-SC,1405947.0,15
X-LC,1047411.0,12
X-SC,823098.0,9
H-MC,395894.0,3
L-SC,178610.0,2
M-LC,130653.0,1
H-LC,77619.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1222937.0      6
           AR         925729.0      5
M-SC       XY24       791099.0      6
H-SC       XR         778517.0      7
X-MC       X40        540180.0      7
X-LC       X40        513816.0      6
X-MC       XY24       487682.0      3
           X40N       397388.0      4
X-SC       X40N       354614.0      4
H-SC       OX40N      320244.0      4
M-SC       OX40N      318649.0      5
X-SC       XY24       302714.0      3
H-SC       SR         290532.0      2
X-LC       X40N       220862.0      2
X-MC       XY25       214283.0      2
H-MC       XY24       206450.0      1
X-LC       XY24       201079.0      2
H-MC       AR         189444.0      2
M-SC       XR         176137.0      2
X-SC       X40        165770.0      2
M-LC       XR         130653.0      1
M-SC       AR         120062.0      2
X-LC       XY25       111654.0      2
L-SC       XR          90622.0      1
           OX40N       87988.0      1
H-LC       AR          77619.0      1
L-MC       XR          49844.0      1
M-MC       XY25        49344.0      1
L-LC       XY25        19790.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
